# Import libraries

In [32]:
import numpy as np
import pandas as pd
import statsmodels.stats as sm
from datetime import datetime
from datetime import timedelta

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly.graph_objects as go

import os
import warnings

warnings.filterwarnings("ignore")
pd.set_option('display.float_format', '{:.2f}'.format)

# Download file

In [33]:
file_id = "1atOZX3YXmxx-_QebbTfndeC6U_DPTL2e" # ID of the file on Google Drive
file_name = 'Updated_data_2021&2022.csv'

%run download.ipynb

# Import data

In [34]:
# File path
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
file_path = os.path.join(parent_dir, 'Data',file_name)

df = pd.read_csv(file_path)

df.head()

MemoryError: Unable to allocate 472. MiB for an array with shape (5, 12364101) and data type object

In [ ]:
df.groupby('BET_ACCOUNT_NUM_HASH').size().sort_values(ascending=False)

## Column type conversion

In [ ]:
# DATE_DIM: datetime
df['DATE_DIM'] = pd.to_datetime(df['DATE_DIM'], format='%Y-%m-%d')

# BET_ACCOUNT_NUM_HASH: string
df['BET_ACCOUNT_NUM_HASH'] = df['BET_ACCOUNT_NUM_HASH'].astype('O')

## New columns

In [ ]:
df['WIN_LOSS_RATIO'] = round(df['DIVIDENDS_PAID'] / df['TOTAL_TURNOVER'],2)
df['AVERAGE_TICKET_VALUE'] = df['TOTAL_TURNOVER'] / df['TICKETS']
df['WIN_LOSS'] = (df['WIN_LOSS_RATIO'] >= 1).astype('int')
df.index = df['DATE_DIM']

df = df.drop(['DATE_DIM', 'DAY_OF_WEEK', 'AGE_BAND', 'GROSS_MARGIN'], axis=1)
df.head()

## 3946065174

In [7]:
df_3946065174 = df[df['BET_ACCOUNT_NUM_HASH'] == 3946065174]
df_3946065174.head()

,BET_ACCOUNT_NUM_HASH,AGE,GENDER,TENURE_IN_DAYS,RESIDENTIAL_STATE,FOB_RACING_TURNOVER,FOB_SPORT_TURNOVER,PARI_RACING_TURNOVER,PARI_SPORT_TURNOVER,TOTAL_TURNOVER,DIVIDENDS_PAID,TICKETS,WIN_LOSS_RATIO,AVERAGE_TICKET_VALUE,WIN_LOSS
18313,3946065174,46.00,M,8464,WA,20.00,NaN,145.00,NaN,165.00,133.57,41,0.81,4.02,0
47909,3946065174,46.00,M,8465,WA,20.00,NaN,147.50,NaN,167.50,137.87,42,0.82,3.99,0
64511,3946065174,46.00,M,8466,WA,15.00,NaN,164.00,NaN,179.00,296.25,46,1.66,3.89,1
75623,3946065174,46.00,M,8467,WA,NaN,NaN,47.00,NaN,47.00,61.15,14,1.30,3.36,1
85801,3946065174,46.00,M,8468,WA,10.00,NaN,122.00,NaN,132.00,0.00,27,0.00,4.89,0


In [8]:
df_3946065174['WIN_LOSS'].mean()

0.3287671232876712

In [9]:
df_3946065174['DIVIDENDS_PAID'].sum() / df_3946065174['TOTAL_TURNOVER'].sum()

0.816051898829559

In [12]:
today = datetime(2022,12,31)
today

datetime.datetime(2022, 12, 31, 0, 0)

In [30]:
def aggregate_by_period(series, func=np.size , from_date = datetime(2022,12,31), days=30):
    start_date = from_date - timedelta(days=days)
    filtered_df = series 
    
    
    return

In [18]:
offset_day = today - timedelta(days=30)

In [ ]:
np.size